In [1]:
import threading
from dobot_api import DobotApiDashboard, DobotApi, DobotApiMove, MyType
from time import sleep
import numpy as np
# 전역 변수 (현재 좌표)
current_actual = None

def connect_robot(ip):
    try:
        dashboard_p = 29999
        move_p = 30003
        feed_p = 30004
        print("연결 설정 중...")
        dashboard = DobotApiDashboard(ip, dashboard_p)
        move = DobotApiMove(ip, move_p)
        feed = DobotApi(ip, feed_p)
        print("연결 성공!!")
        return dashboard, move, feed
    
    except Exception as e:
        print("연결 실패")
        raise e
    
def robot_clear(dashboard : DobotApiDashboard):
    dashboard.ClearError()

def robot_speed(dashboard : DobotApiDashboard, speed_value):
    dashboard.SpeedFactor(speed_value)

def gripper_DO(dashboard : DobotApiDashboard, index, status):
    dashboard.ToolDO(index, status)
    
def get_Pose(dashboard : DobotApiDashboard):
    dashboard.GetPose()

def run_point(move: DobotApiMove, point_list: list):
    move.MovL(point_list[0], point_list[1], point_list[2], point_list[3])

def get_feed(feed: DobotApi):
    global current_actual
    hasRead = 0

    while True:
        data = bytes()
        while hasRead < 1440:
            temp = feed.socket_dobot.recv(1440 - hasRead)
            if len(temp) > 0:
                hasRead += len(temp)
                data += temp
        hasRead = 0
        a = np.frombuffer(data, dtype=MyType)

        if hex((a['test_value'][0])) == '0x123456789abcdef':
            current_actual = a["tool_vector_actual"][0]
        # Refresh Properties
        sleep(0.001)

def wait_arrive(point_list):
    global current_actual
    while True:
        is_arrive = True
        if current_actual is not None:
                for index in range(4):
                    if (abs(current_actual[index] - point_list[index]) > 1):
                        is_arrive = False
                if is_arrive:
                    return
        sleep(0.001)

# 입력 파라미터
ip = "192.168.1.6"
gripper_port = 1
speed_value = 30
# Robot의 IP 주소
# 그리퍼 포트 번호
# 로봇 속도 (1~100 사이의 값 입력)
# 로봇이 이동하고자 하는 좌표 (x, y, z, yaw) unit : mm, degree
point_home = [245, 5, 15, 0]


# 로봇 연결
dashboard, move, feed = connect_robot(ip)
dashboard.EnableRobot()
print("이제 로봇을 사용할 수 있습니다!")
# 쓰레드 설정
feed_thread = threading.Thread(target=get_feed, args=(feed,))
feed_thread.setDaemon(True)
feed_thread.start()

import cv2
from time import sleep
import numpy as np

positions = []
box = []

def main():
    cap = cv2.VideoCapture(2)

    #global x, y, color_name
    is_first_circle = True  # 첫 번째 원인지 여부를 나타내는 변수
    while True:
        # 프레임 읽기
        ret, frame = cap.read()
        # 프레임이 성공적으로 읽혔는지 확인
        if not ret:
            print("Error: Could not open camera")
            break
        
        key = cv2.waitKey(1)

        # 필터링
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        enhanced = cv2.equalizeHist(gray)
        blurred = cv2.GaussianBlur(enhanced, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        #cv2.imshow("filter", edges)
        # 원 검출
        circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=30,
                                param1=20,
                                param2=20,
                                minRadius=5,
                                maxRadius=12)
        if circles is not None:
            # 검출된 원의 중심과 반지름 추출
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, r) in circles:
                # 반지름이 20 이하인 원을 초록색 원으로 그리기
                cv2.circle(frame, (x, y), r, (0, 255, 0), 2)
                cv2.circle(frame, (x, y), 15, (255, 0, 0), 2)
                cv2.circle(frame, (x, y), 40, (255, 0, 0), 2)
                # 반지름이 15와 40인 두 원 사이의 픽셀 색상 검출
                color = frame[y+20,x]  # 반지름 15인 원의 픽셀 색상
                b, g, r = color
                color_name = 'none'
                if g>b and g>r:
                #if b>170 and g>150 and r>150:
                    color_name = 'green'
                elif r>g and r>b and g<100:
                #elif b < 100 and g<100 and r>130:
                    color_name = 'red'
                elif b<g and b<r:
                #elif b<170 and g>200 and r>200:
                    color_name = 'yellow'
                # 반지름 40인 원의 픽셀 색상
                #print("Color 1:", color)
                cv2.putText(frame, f"{[x, y]}, {color_name}", (x - 50, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
                x, y, r = circles[0]
                if is_first_circle:
                    positions.append([y, x])  # 첫 번째 원의 좌표를 리스트에 추가
                    is_first_circle = False  # 첫 번째 원 처리가 끝났으므로 변수 업데이트

        # box = positions[0]
        print(positions)
        # print(color_name)
        # print(box)
        
        # print(box)
        # 결과 영상 출력
        cv2.imshow("Circle Detection", frame)
        if len(positions) > 0:
            # sleep(1)
            cap.release()
            cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

# 카메라 좌표 (x,y)
camera_coords = np.array([[40, 40], [250, 40], [454, 40],
                          [98, 322], [250,322], [446,322],
                          [40, 616], [24,616], [45, 616]])

# 로봇 좌표 (x,y)
robot_coords = np.array([[190, -131], [285, -123], [386, -130],
                         [215, 6], [284, 8], [371, 10],
                         [187, 141], [278, 141], [371, 142]])

# 테스트 카메라 좌표 (x,y)
test_coords = np.array(positions)

# 변환 매트릭스 계산
transformation_matrix, _ = cv2.estimateAffinePartial2D(camera_coords, robot_coords)

# 카메라 좌표를 로봇 좌표로 변환
transformed_coords = cv2.transform(np.array([test_coords], dtype=np.float32), transformation_matrix)

print("변환된 로봇 좌표:\n", transformed_coords.squeeze())

ex_point = transformed_coords[0].tolist()
ex_point = ex_point[0]
ex_point.append(0)
ex_point.append(0)

print('ex_point:', ex_point)

연결 설정 중...
연결 성공!!
Send to 192.168.1.6:29999: EnableRobot()
Receive from 192.168.1.6:29999: 0,{},EnableRobot();
이제 로봇을 사용할 수 있습니다!


/tmp/ipykernel_8616/397035634.py:87: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  feed_thread.setDaemon(True)


[[208, 246]]
Error: Could not open camera
변환된 로봇 좌표:
 [264.81158  -27.411575]
ex_point: [264.81158447265625, -27.411575317382812, 0, 0]


In [2]:
first = -56.599354
second = -38.825260
third = -22.097727

point_red = [248.731104,-109.077538,-72.123154,0,0.000000,0.000000]
point_green = [333.788251,3.451116,-72.212730,0,0.000000,0.000000]
point_yellow = [246.725365,108.133964,-72.442833,0,0.000000,0.000000]
run_point(move, point_home)

def move_gripper(start_point, start_height, end_point, end_height):
    start_point[2] = start_height
    run_point(move, start_point)
    wait_arrive(start_point)
    gripper_DO(dashboard, gripper_port, 1)

    run_point(move, point_home)
    wait_arrive(point_home)

    end_point[2] = end_height
    run_point(move, end_point)
    wait_arrive(end_point)
    gripper_DO(dashboard, gripper_port, 0)

    run_point(move, point_home)
    wait_arrive(point_home)

# ex_point = [216.486225,4.014995,-22.5,115.015442]

move_gripper(ex_point, third, point_red, first) # 쌓여있는 곳 갔다 시작 지점으로
move_gripper(ex_point, second, point_yellow, first)
move_gripper(ex_point, first, point_green, first)

move_gripper(point_yellow, first, point_green, second)
move_gripper(point_red, first, point_green, third)
    


Send to 192.168.1.6:30003: MovL(245.000000,5.000000,15.000000,0.000000)
Receive from 192.168.1.6:30003: 0,{},MovL(245.000000,5.000000,15.000000,0.000000);
Send to 192.168.1.6:30003: MovL(264.811584,-27.411575,-22.097727,0.000000)
Receive from 192.168.1.6:30003: 0,{},MovL(264.811584,-27.411575,-22.097727,0.000000);
Send to 192.168.1.6:29999: ToolDO(1,1)
Receive from 192.168.1.6:29999: 0,{},ToolDO(1,1);
Send to 192.168.1.6:30003: MovL(245.000000,5.000000,15.000000,0.000000)
Receive from 192.168.1.6:30003: 0,{},MovL(245.000000,5.000000,15.000000,0.000000);
Send to 192.168.1.6:30003: MovL(248.731104,-109.077538,-56.599354,0.000000)
Receive from 192.168.1.6:30003: 0,{},MovL(248.731104,-109.077538,-56.599354,0.000000);
Send to 192.168.1.6:29999: ToolDO(1,0)
Receive from 192.168.1.6:29999: 0,{},ToolDO(1,0);
Send to 192.168.1.6:30003: MovL(245.000000,5.000000,15.000000,0.000000)
Receive from 192.168.1.6:30003: 0,{},MovL(245.000000,5.000000,15.000000,0.000000);
Send to 192.168.1.6:30003: MovL(2

In [3]:
# robot_clear(dashboard)

In [4]:
dashboard.DisableRobot()

Send to 192.168.1.6:29999: DisableRobot()
Receive from 192.168.1.6:29999: 0,{},DisableRobot();


'0,{},DisableRobot();'